In [1]:
import requests
from bs4 import BeautifulSoup
import time
import os

In [2]:
headers = {
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36",
    "referer":"https://icook.tw/categories?ref=icook-footer"
}

In [ ]:
url = "https://icook.tw/categories"

res = requests.get(url=url,headers=headers)
soup = BeautifulSoup(res.text, "html.parser")
tmp_soup = soup.select("a[class='categories-all-child-link']")
for i in tmp_soup:
    category = i.text
    if not os.path.exists("./{}".format(category.replace("\n","").replace("/",""))"):
        os.mkdir("./{}".format(category.replace("\n","").replace("/","")))
    category_link = "https://icook.tw/" + i["href"]
    print(category)
    print(category_link)
    
    count_res = requests.get(url=category_link, headers=headers)
    count_soup = BeautifulSoup(count_res.text, "html.parser")
    title_count = count_soup.select("span[class='browse-title-count']")[0].text
    print(title_count)
    
    
    page = 1
    while True:
        food_res = requests.get(url=category_link + "?page={}".format(page), headers=headers)
        food_soup = BeautifulSoup(food_res.text, "html.parser")
        no_food = food_soup.select("title")[0].text
        if no_food == "愛料理上找不到您要的頁面喔～":
            break
        tmp_food_soup = food_soup.select("li[class='browse-recipe-item']")
        title_count = food_soup.select("span[class='browse-title-count']")[0].text
        print(title_count)
        
        for i in tmp_food_soup:
            food_title = i.a["aria-label"][2:].replace("/","")
            food_link = "https://icook.tw/" + i.a["href"]
            food_img = i.select("img")[0]["data-src"]
            print(food_title)
            print(food_link)
            print(food_img)
            
            recipe_res = requests.get(url=food_link, headers=headers)
            recipe_soup = BeautifulSoup(recipe_res.text, "html.parser")
            ingredient_group = recipe_soup.select("a[class='ingredient-search']")
            total_ingredient = ""
            for tmp_ingredient in ingredient_group:
                ingredient = tmp_ingredient.text
                print(ingredient)
                total_ingredient += ingredient +"\n"
        
        
            with open("./{}/{}.txt".format(category.replace("\n","").replace("/",""),food_title), "w", encoding="utf-8") as f:
                f.write(food_title +"\n"+ food_link +"\n"+ total_ingredient +"\n"+ food_img)

        page += 1
    
        if page % 3 == 0:
            time.sleep(10)